In [ ]:
### vscode에서는 실행불가. jupyter를 이용하는 것을 권장 ###

# 라이브러리
import pandas as pd
import plotly.express as px
from prophet import Prophet
import glob
from natsort import natsorted

import warnings
warnings.filterwarnings(action='ignore')

files = glob.glob('../Naver_Search_Amount/data/result/search_result_absolute*.csv') #최신파일 읽어오기
recent_file = natsorted(seq=files, reverse=True)[0]
df_anomaly = pd.read_csv(recent_file, encoding='cp949')
df_anomaly = df_anomaly[df_anomaly.columns[df_anomaly.apply(lambda x: len(x[x.notnull().values]) > 30).values == True]] # 값이 30개 이상인 항목만
weight = 1.5  #가중치 직접 설정가능 (오차 허용범위(높을 수록 둔감))
min_value = 10000

def visualization(df_anomaly):
    search_key = input('검색어를 입력해주세요: ')
    if search_key not in df_anomaly.columns:
        return print('해당 검색어는 이상감지 목표 검색어에 해당하지 않습니다. config 파일에 추가 후 재실행 해주십시오.')
    new_df = df_anomaly[['날짜', search_key]]
    new_df['날짜'] = new_df['날짜'].astype('datetime64[ns]')
    new_df = new_df.set_index('날짜')
    new_df_prophet = new_df.reset_index()[['날짜', new_df.columns[0]]].rename({'날짜':'ds', new_df.columns[0]:'y'}, axis='columns')
    m = Prophet()#changepoint_range=0.95, changepoint_prior_scale=0.05) #holidays=holidays)
    m.add_country_holidays(country_name='KR')
    m.fit(new_df_prophet)
    future = m.make_future_dataframe(periods=365, freq='D')
    forecast = m.predict(future)
    forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(10)
    result = pd.concat([new_df_prophet.set_index('ds')['y'], forecast.set_index('ds')[['yhat','yhat_lower','yhat_upper']]], axis=1)
    result['error'] = result['y'] - result['yhat'] #실제값 - 예측값범위
    result['uncertainty'] = result['yhat_upper'] - result['yhat_lower'] #범위, 불확실성
    # 본래 abs를 붙여 낮게 나온 이상치도 출력하는 코드였지만, 검색어 특성상 높게 나온 지점만 산출하기 위해 abs 삭제
    #비율에 따라, 검색량이 절대적으로 적은(일정 수치 이하) 경우는 가중치를 바꾸도록??
    result['anomaly'] = result.apply(lambda x: 'Yes' if((x['error']) > weight*x['uncertainty']) else 'No', axis = 1) # 하한선 지정을 제외했다. 원한다면 error을 abs(절대값)로
    result['anomaly'] = result.apply(lambda x: 'Low' if (x['anomaly']=='Yes') & (x['y'] < min_value) else 'Yes' if (x['anomaly']=='Yes') & (x['y'] >= min_value) else 'No', axis=1) #10000 이하는 Low로
    #visualize the anomaly data
    fig = px.scatter(result.reset_index(), x='ds', y='y', color='anomaly', 
                     color_discrete_map={'Yes': '#EF553B','No': '#636EFA', 'Low': '#00CC96'}, title=new_df.columns[0])    
    #slider
    fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                    dict(count=1, label='1y', step="year", stepmode="backward"),
                    dict(count=2, label='3y', step="year", stepmode="backward"),
                    dict(count=2, label='5y', step="year", stepmode="backward"),
                    dict(step="all")
            ])
        )
    )
    fig.show()

visualization(df_anomaly)